In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/alexanderdahm/Documents/GitHub/mlfs-book-proj
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

2025-12-28 13:10:19,913 WARNING: DeprecationWarning: 'parseString' deprecated - use 'parse_string'

2025-12-28 13:10:19,913 WARNING: DeprecationWarning: 'resetCache' deprecated - use 'reset_cache'

2025-12-28 13:10:19,942 WARNING: DeprecationWarning: 'enablePackrat' deprecated - use 'enable_packrat'

2025-12-28 13:10:19,957 WARNING: In /Users/alexanderdahm/miniconda3/envs/aq/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'parseString' deprecated - use 'parse_string'
2025-12-28 13:10:19,957 WARNING: In /Users/alexanderdahm/miniconda3/envs/aq/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'resetCache' deprecated - use 'reset_cache'


In [3]:
class Sensor:
    def __init__(self, name, city, lat, lon, csv=""):
        self.name = name
        self.country = "sweden"
        self.city = city
        self.street = city
        self.url = ""  # no explicit AQICN URL provided for these cities
        self.lat = float(lat)
        self.lon = float(lon)
        self.csv = csv

cities = [
    {"name": "flasjon", "lat": 62.760350390111626, "lon": 13.715986496712969},
    {"name": "hudiksvall", "lat": 61.790862930411194, "lon": 17.15754858778168},
    {"name": "ange", "lat": 62.54989082316923, "lon": 15.751547550392734},
    {"name": "solleftea", "lat": 63.159587742988755, "lon": 17.2655114712721},
    {"name": "umea", "lat": 63.81702480736613, "lon": 20.18691175826482},
]

sensorList = []
for idx, c in enumerate(cities):
    sensorList.append(Sensor(f"sensor{idx}", c["name"], c["lat"], c["lon"], ""))
    

   

for sensor in sensorList:
    print(f"Processing sensor: {sensor.name} located at {sensor.street}, {sensor.city}, {sensor.country} with coordinates ({sensor.lat}, {sensor.lon}), csv path {sensor.csv}")


Processing sensor: sensor0 located at flasjon, flasjon, sweden with coordinates (62.760350390111626, 13.715986496712969), csv path 
Processing sensor: sensor1 located at hudiksvall, hudiksvall, sweden with coordinates (61.790862930411194, 17.15754858778168), csv path 
Processing sensor: sensor2 located at ange, ange, sweden with coordinates (62.54989082316923, 15.751547550392734), csv path 
Processing sensor: sensor3 located at solleftea, solleftea, sweden with coordinates (63.159587742988755, 17.2655114712721), csv path 
Processing sensor: sensor4 located at umea, umea, sweden with coordinates (63.81702480736613, 20.18691175826482), csv path 


## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [4]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
"""location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str"""
today = datetime.date.today()

2025-12-28 13:10:20,085 INFO: Initializing external client
2025-12-28 13:10:20,085 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-28 13:10:36,710 INFO: Closing external client and cleaning up certificates.
2025-12-28 13:10:36,744 INFO: Connection closed.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1290388/credentials). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":110021,"usrMsg":"Failed to download credentials for projectId: 1290388","devMsg":"jakarta.ejb.EJBTransactionRolledbackException: Exception thrown from bean: jakarta.ejb.EJBTransactionRolledbackException: Exception thrown from bean: jakarta.ejb.EJBException: An error has occurred.","errorMsg":"Failed to download."}', error code: 110021, error msg: Failed to download., user msg: Failed to download credentials for projectId: 1290388

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='energy_price',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [ ]:
# Get the historical air quality, so we can use it to get roll3 value of today
aq_history = air_quality_fg.read(online=False)[["date", "pm25", "country", "city", "street", "url"]]
aq_history["date"] = pd.to_datetime(aq_history["date"], utc=True).dt.tz_convert(None)

# List to store new values
aq_rows = []
# Collect per-city daily weather dfs to merge into wide format (same as file 1)
all_weather_data = []

# Loop over each sensor and add their daily data
for sensor in sensorList:
    country = sensor.country
    city = sensor.city
    street = sensor.street
    aqicn_url = sensor.url
    latitude = sensor.lat
    longitude = sensor.lon

    # Get today air quality
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    aq_today_df["date"] = pd.to_datetime(aq_today_df["date"], utc=True).dt.tz_convert(None)

    # Filter historical data to this sensor only: Use city and street as unique identifiers.
    aq_history_sensor = aq_history[(aq_history["city"] == city) & (aq_history["street"] == street)]

    # Concat historic and daily air quality data for a specific sensor
    aq_all = pd.concat([aq_history_sensor, aq_today_df], ignore_index=True)
    aq_all = aq_all.sort_values("date").reset_index(drop=True)

    # Keep only today's data and drop if nan
    aq_today_data = aq_all[aq_all["date"] == aq_today_df["date"].iloc[0]]
    if not aq_today_data.empty:
        aq_rows.append(aq_today_data)

    # Get weather data (daily at ~12:00), then rename columns with city suffix to produce wide format
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index("date")

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_df = hourly_df.between_time('11:59', '12:01')
    daily_df = daily_df.reset_index()
    daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
    daily_df['date'] = pd.to_datetime(daily_df['date'])
    # Rename columns to include city name (exclude 'date')
    daily_df = daily_df.rename(columns={col: f"{col}_{city}" for col in daily_df.columns if col != "date"})
    all_weather_data.append(daily_df)

# Upload new data to feature store
if aq_rows:
    aq_insert_df = pd.concat(aq_rows, ignore_index=True)
    air_quality_fg.insert(aq_insert_df)

if all_weather_data:
    combined_weather_df = all_weather_data[0]
    for df in all_weather_data[1:]:
        combined_weather_df = pd.merge(combined_weather_df, df, on="date", how="outer")
    weather_fg.insert(combined_weather_df, wait=True)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
